## Homework #6. Behaviour exploration improvement
#### Author: Kateryna Steshenko
#### Total time spent on h/w (in minutes): 300

Структуризація датасету у формат таблиць за допомогою pandas:

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [17]:
DIALOGS_MERGED_DATA_PATH = "dialogs_data_all.csv"
DIALOGS_META_MERGED_DATA_PATH = "dialogs_users_all.csv"

In [21]:
def convertFrom(d):
    res = []
    for t in range(d.shape[0]):
        listId = list(map(int, re.findall(r'\d+', str(d["from_id"].iloc[[t]])[len(str(t)):])))
        listUser = list(map(int, re.findall(r'\d+',str(d["users"].iloc[[t]])[len(str(t)):])))
        if len(listId) > 0 and len(listUser) > 0:
            from_id = listId[0]
            user = listUser[0]
            res.append(from_id == user)
        else: res.append(False)
    return res


In [19]:
df = pd.read_csv(DIALOGS_MERGED_DATA_PATH)
df_meta = pd.read_csv(DIALOGS_META_MERGED_DATA_PATH)
mergedPrivate = df.merge(df_meta[df_meta["type"] == "Private dialog"],  how='inner', on='dialog_id')
fakeMergedGroup = df.merge(df_meta[df_meta["type"] == "Group"],  how='inner', on='dialog_id')


C:\Users\User\AppData\Local\Temp\ipykernel_21192\1607247470.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DIALOGS_MERGED_DATA_PATH)


In [22]:
fakeMergedGroup["res"] = convertFrom(fakeMergedGroup)  #it takes about 30 minutes
mergedGroup = fakeMergedGroup[fakeMergedGroup["res"] == True]
mergedGroup = mergedGroup.drop("res")

KeyboardInterrupt: 

In [ ]:
mergedDf = pd.concat([mergedGroup, mergedPrivate])
mergedDf.shape

In [ ]:
ukr_stopwords = []
with open("ukrainian_stopwords.txt", 'r', encoding="utf-8") as f:
    ukr_stopwords = [line.strip() for line in f]

## Task
## Data Analysis


1. В певний момент часу моя подруга переїхала в Канаду. Дослідження того як змінилось інтенсивність наших переписок

In [ ]:
def convert_times(d):
    mylist = []
    for t in range(len(d["date"].tolist())):
        msgdate = d["date"].loc[d.index[t]][:10]
        mylist.append(pd.to_datetime(msgdate))
    return mylist

In [ ]:
myUserId = int(input("Input your ID ")) #lnput your user ID like 111111111 How to find look prepare.ipynb
yourFriendName = input("Input Your chat name of partner or friend name dialog (your friend firstname and second name in TG) ")

In [ ]:
mfdf = df[df["dialog_id"] == int(df_meta[df_meta["name"] == f"{yourFriendName}" ]['dialog_id']) ]


dfr = mfdf[mfdf["to_id"] == myUserId]
dfr["DMY"] = convert_times(dfr)
dfs = mfdf[mfdf["to_id"] != myUserId]
dfs["DMY"] = convert_times(dfs)
dfr.groupby("DMY")["DMY"].count().plot(label = "received")
dfs.groupby("DMY")["DMY"].count().plot(label = "sent")
plt.axvline(x = "2021-12-26", color = 'b', label = "day of departure" )
plt.axvline(x = "2022-02-24", color = 'r', label = "invasion")
plt.title("Intensity of messaging with my friend")
plt.legend()
plt.show()



2. Оскільки часовий пояс в Канаді сильно віддмінний від українського, переписуватись ми стали в інший час. 2 графіки того з часом відправлення повідомлень відносно наших часових поясів

In [ ]:
def convert_hours(d, n):
    mylist = []
    for t in range(len(d["date"].tolist())):
        msghour = int(d["date"].loc[d.index[t]][11:13]) + n
        if msghour > 23:
            msghour -= 24
        if msghour < 0:
            msghour += 24
        mylist.append(msghour)
    return mylist

mfdf1 = mfdf[mfdf["date"] < "2021-12-26 00-00-00+00:00"]
mfdf2 = mfdf[mfdf["date"] >= "2021-12-26 00-00-00+00:00"]

dfrh1 = mfdf1[mfdf1["to_id"] == myUserId ]
dfrh1["DMY"] = convert_hours(dfrh1, 2)
dfsh1 = mfdf1[mfdf1["to_id"] != myUserId]
dfsh1["DMY"] = convert_hours(dfsh1, 2)
dfrh2 = mfdf2[mfdf2["to_id"] == myUserId]
dfrh2["DMY"] = convert_hours(dfrh2, -5)
dfsh2 = mfdf2[mfdf2["to_id"] != myUserId]
dfsh2["DMY"] = convert_hours(dfsh2, 2)


fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
dfrh1.groupby("DMY")["DMY"].count().plot( ax=ax1)
dfsh1.groupby("DMY")["DMY"].count().plot( ax=ax1)
dfrh2.groupby("DMY")["DMY"].count().plot( ax=ax2)
dfsh2.groupby("DMY")["DMY"].count().plot( ax=ax2)
plt.sca(ax1)
plt.title("Messaging time before departure")
plt.sca(ax2)
plt.title("Messaging time after departure")
plt.show()


3. Залежність між довжиною слів та їх використанням.

In [ ]:
words = {}
sent = df[df["from_id"] == f'PeerUser(user_id={myUserId})']
recev = df[df["from_id"] != f'PeerUser(user_id={myUserId})']
for ms in list(sent["message"]):
    mslist = re.sub("[^a-zA-Z\u0401-\u04f9]+", " ",str(ms).lower()).replace("[^a-zA-Z0-9]+", ' ').split(' ')
    for w in mslist:
        if len(w) > 0:
            if len(w) in words:
                words[len(w)] += 1
            else:
                words[len(w)] = 1

wordnumdf = pd.DataFrame([(k, v) for k, v in words.items()], columns = ["length", "times"])
wordnumdf[wordnumdf["length"]<=20].sort_values(by=['length']).plot(x = "length", y = "times", color = "orange", kind = 'bar')
plt.title("Dependency between words length and usage in sent message")
rwords = {}
for ms in list(recev["message"]):
    mslist = re.sub("[^a-zA-Z\u0401-\u04f9]+", " ",str(ms).lower()).replace("[^a-zA-Z0-9]+", ' ').split(' ')
    for w in mslist:
        if len(w) > 0:
            if len(w) in rwords:
                rwords[len(w)] += 1
            else:
                rwords[len(w)] = 1

rwordnumdf = pd.DataFrame([(k, v) for k, v in rwords.items()], columns = ["length", "times"])
rwordnumdf[rwordnumdf["length"]<=20].sort_values(by=['length']).plot(x = "length", y = "times", kind = 'bar')
plt.title("Dependency between words length and usage in receive message")
plt.show()

4. Інтенсивність спілкування в приватних повідомленнях та групах в часовому проміжку. Тобто кількість повідомлень за 1 місяць в групах та приватних діалогах

In [ ]:
def convert_month(d):
    mylist = []
    for t in range(len(d["date"].tolist())):
        mylist.append(pd.to_datetime(d["date"].loc[d.index[t]][:7]))
    return mylist


groups = mergedGroup
groups["DMY"] = convert_month(groups)
privates = mergedPrivate
privates["DMY"] = convert_month(privates)
groups.groupby("DMY")["DMY"].count().plot(color = 'g', style="--", label = "Group")
privates.groupby("DMY")["DMY"].count().plot( color = 'r', style="--", label ="Private dialog")
plt.axvline(x = "2022-02-24", color = 'black', label = "invasion") #invasion"
plt.title("Intensity of messaging")
plt.legend()
plt.show()

5. Тенденція використання типових слів про навчання та Пласт

In [ ]:
studyList = ["лаб", "дз", "домашк", "пар", "задавали", "глиб", "роблю", "здач", "кр", "колок", "трим", "матан", "урок", "дискретка", "прога", "проект", "код", "формул", "таск", "сесі", "індивідуал", "зно", "мобільн"]
plastList = ["вкладк", "сходин", "табір", "збірк", "акці", "вишкіл", "гутірк", "проба", "пробу", "провід", "курінь", "бунч", "комендант", "інструкт" "гурток", "виховн", "зв'язков", "впр", "змаг", "мандр", "рейд"]
deadline = ["дедлайн", "реченець", "deadline", "термін"]
def searcher(d, searchlist):
    column = []
    for ms in list(d["message"]):
        res = 0
        for w in searchlist:
            if w in str(ms).lower():
                res += 1
        column.append(res)
    return column


fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

ddl = pd.DataFrame()
ddl["DMY"] = convert_month(df)
ddl["Numb"] = searcher(df, deadline)
ddl.groupby("DMY")["Numb"].sum().plot(ax = ax1, color = "black")
plast = pd.DataFrame()
plast["DMY"] = convert_month(df)
plast["Numb"] = searcher(df, plastList)
plast.groupby("DMY")["Numb"].sum().plot(ax = ax2, color = "green", label = "Plast")
study = pd.DataFrame()
study["DMY"] = convert_month(df)
study["Numb"] = searcher(df, studyList)
study.groupby("DMY")["Numb"].sum().plot(ax = ax2, color = "blue", label = "study")
plt.sca(ax1)
plt.title("Using words deadline")
plt.sca(ax2)
plt.title("Using words about Plast and study")
plt.legend()
plt.show()

6. Тенденція отримання повідомлень від одногрупників в особистих та групах

In [ ]:
fst = input("Input name of classmates group")
snd = input("Input other name of classmates group")
trd = input("Input other name of classmates group")

In [ ]:
def getUsersIdlist(groupName):
    userlist = []
    d = df_meta[df_meta["name"] == groupName]["users"]

    for t in range(d.shape[0]):
        userlist.append(d.iloc[[t]].values[0])
    return userlist


myClassMates = getUsersIdlist(fst) + getUsersIdlist(snd)+ getUsersIdlist(trd)
condition = mergedDf.isin({"users": myClassMates})["users"]
classMatesDF = mergedDf[condition]
classMatesDF = classMatesDF[classMatesDF["from_id"] == f'PeerUser(user_id={myUserId})']

cGroup = classMatesDF[classMatesDF["type_y"] == "Group"]
cPrivate = classMatesDF[classMatesDF["type_y"] == "Private dialog"]
cGroup["DMY"] = convert_month(cGroup)
cPrivate["DMY"] = convert_month(cPrivate)

cGroup.groupby("DMY")["DMY"].count().plot(label = "groups")
cPrivate.groupby("DMY")["DMY"].count().plot(label = "private")
plt.title("Messaging with classmates")
plt.legend()


7. Доступ до спілкування з якою кількістю людей через різні типи чатів

P, G, C - Доступ тільки через Приватні, Групи і Канали відповідно
PG, PC, GC - Доступ тільки через Приватні та Групи, Приватні та Канали, Групи та Канали відповідно
PGC - Доступ через всі види зв'язку

In [ ]:
def get_variable_name(variable):
    globals_dict = globals()
    return [var_name for var_name in globals_dict if globals_dict[var_name] is variable][0], variable.shape[0]

def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i,y[i],y[i])

dff = pd.DataFrame()
dff["users"]=df_meta["users"]
dff["type"]=df_meta["type"]
noDuplDF=dff.drop_duplicates()

allP = pd.DataFrame();allG = pd.DataFrame();allC = pd.DataFrame();
allP["users"] = noDuplDF[noDuplDF["type"] == "Private dialog"]["users"]
allG["users"] = noDuplDF[noDuplDF["type"] == "Group"]["users"]
allC["users"] = noDuplDF[noDuplDF["type"] == "Channel"]["users"]

PG = pd.merge(allP, allG, how='inner', on = ["users"])
PC = pd.merge(allP, allC, how='inner', on = ["users"])
GC = pd.merge(allG, allC, how='inner', on = ["users"])

PGC = pd.merge(GC, pd.merge(PG, GC, how='inner', on = ["users"]), how='inner', on = ["users"])

allPGC = pd.concat([pd.concat([PG, PC]).drop_duplicates(), GC]).drop_duplicates()

PG = PG.merge(PGC,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
PC = PC.merge(PGC,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
GC = GC.merge(PGC,indicator = True, how='left').loc[lambda x : x['_merge']!='both']


P = allP.merge(allPGC,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
G = allG.merge(allPGC,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
C = allC.merge(allPGC,indicator = True, how='left').loc[lambda x : x['_merge']!='both']

types = [get_variable_name(P),
         get_variable_name(G),
         get_variable_name(C), 
         get_variable_name(PG), 
         get_variable_name(PC), 
         get_variable_name(GC), 
         get_variable_name(PGC)
        ]
xticks = []
yl = []
for i in range(len(types)):
    xticks.append(types[i][0])
    yl.append(types[i][1])
x = np.arange(len(xticks))
y = np.array(yl)
plt.xticks(x, xticks)
plt.bar(x, y)
addlabels(x, y)
plt.title("Ways to communicate with users")
plt.show()    


8. Середня інтенсивність серед топ 20 приватних чатів з найбільшою кількістю повідомлень. Інтенсивність обраховується за формулою: І = (к-ть повідомлень)/(день останнього повідомлення - день 1 повідомлення). Оскільки до обрахунків увійшди тільки найбільші чати, проблеми із діленням на 0 не має виникнути.

In [ ]:
import seaborn as sns

def findMinDate(d):
    mylist = []
    for t in range(d.shape[0]):
        current = mergedPrivate[mergedPrivate["name"] == d["name"].values[t]]
        mylist.append(pd.to_datetime(min(current["date"])[:10]))
    return mylist
def findMaxDate(d):
    mylist = []
    for t in range(d.shape[0]):
        current = mergedPrivate[mergedPrivate["name"] == d["name"].values[t]]
        mylist.append(pd.to_datetime(max(current["date"])[:10]))
    return mylist
def getDep(d):
    minList = findMinDate(d)
    maxList = findMaxDate(d)
    sizeList = d["size"].values
    res = []
    for i in range(d.shape[0]):
        res.append(sizeList[i]/(maxList[i] - minList[i]).days)
    return res

newdf = pd.DataFrame()
newdf["size"] = mergedPrivate.groupby(["name"])["name"].count().sort_values(ascending=False).head(20)
newdf.reset_index(inplace=True)
newdf = newdf.rename(columns = {'index':'name'})
newdf["res"] = getDep(newdf)
newdf= newdf.sort_values(by=['res'])
plt.xticks(rotation=45, ha='right')
sns.swarmplot(x="name", y="res",  data=newdf)
plt.title("Intensity of messaging") 


9. Візуалізація 15 найбільших чатів по кількості учасників.

In [ ]:
groupDF = pd.DataFrame()
groupDF["user_amount"] = df_meta[df_meta["type"]=="Group"].groupby(["name"])["name"].count().sort_values(ascending=False).head(15)
groupDF.reset_index(inplace=True)
groupDF = groupDF.rename(columns = {'index':'name'})

xticks = groupDF["name"].values
y = np.array(groupDF["user_amount"].values)
x = np.arange(len(xticks))
plt.xticks(x, xticks)
plt.bar(x, y)
addlabels(x, y)
plt.xticks(rotation=45, ha='right')
plt.title("The biggest droups") 
plt.show()  

10. Візуалізація приватних чатів з найбільшою кількістю повідомлень

In [ ]:
privateDF = pd.DataFrame()
privateDF["msg_amount"] = mergedPrivate.groupby(["name"])["name"].count().sort_values(ascending=False).head(10)
privateDF.reset_index(inplace=True)
privateDF = privateDF.rename(columns = {'index':'name'})

xticks = privateDF["name"].values
y = np.array(privateDF["msg_amount"].values)
x = np.arange(len(xticks))
plt.xticks(x, xticks)
plt.bar(x, y)
addlabels(x, y)
plt.xticks(rotation=45, ha='right')
plt.title("The biggest private dialogs") 
plt.show()  

11. Юзери з прихованим номером телефну/контакти

In [ ]:
def getNumbersAmount():
    d = pd.DataFrame()
    d["users"] = df_meta["users"].drop_duplicates()
    res = 0
    for user in d["users"].values:
        listUser = list(map(str, re.findall(r'\d+', user)))
        if len(listUser) > 0:
            phone = listUser[len(listUser) - 1]
            if len(phone) == 12 and "380" in phone:
                res += 1
    return res

def addpersent(x,y, alu, l):
    for i in range(len(x)):
        if l == 0:
            numb = int(y[i]/alu*100)
        else:
            numb = round(y[i]/alu*100, l)
        if numb != 0:
            plt.text(i,y[i],f'{numb}%')
        
        
allus = df_meta["users"].drop_duplicates().size
contacts = getNumbersAmount()
nocontacts =  allus - contacts

xticks = ["Відомий", "Прихований"]
y = np.array([contacts, nocontacts])
x = np.arange(len(xticks))
plt.xticks(x, xticks)
plt.bar(x, y)
addpersent(x,y, allus, 1)
plt.title("Номер телефону")
plt.show() 

12. Який відсоток юзерів мають нік, прізвище, ім'я

In [ ]:
def getNoNameAmount(category):
    d = pd.DataFrame()
    d["users"] = df_meta["users"].drop_duplicates()
    res = 0
    for user in d["users"].values:
        user = user.replace('"', "'")
        if f"{category}': None" in user:
            res += 1
    return res
        
    
nickname = allus - getNoNameAmount("username") 
firstname = allus - getNoNameAmount("first_name") 
lastname = allus - getNoNameAmount("last_name") 

xticks = ["User name", "First name", "Last name"]
y = np.array([nickname, firstname, lastname])
x = np.arange(len(xticks))
plt.xticks(x, xticks)
plt.bar(x, y)
addpersent(x,y, allus, 1)
plt.title("Users have:")
plt.show() 

13. Дослідити час відправлення мною повідомлень впродовж доби

In [ ]:
otherMsg = df[df["from_id"] != f"PeerUser(user_id={myUserId})"]
otherMsg["DMY"] = convert_hours(otherMsg, 2)
counts = pd.DataFrame()
counts["amount"] = otherMsg.groupby("DMY")["DMY"].count()
counts["amount"].plot(kind = "bar", label = "recived")
counts.reset_index(inplace=True)
y = np.array(counts["amount"].values.tolist())
x = np.array(counts["DMY"].values.tolist())
addpersent(x,y, otherMsg.shape[0], 0)


myMsg = df[df["from_id"] == f"PeerUser(user_id={myUserId})"]
myMsg["DMY"] = convert_hours(myMsg, 2)
counts = pd.DataFrame()
counts["amount"] = myMsg.groupby("DMY")["DMY"].count()
counts["amount"].plot(kind = "bar", color = "orange",  label = "sent")
counts.reset_index(inplace=True)
y = np.array(counts["amount"].values.tolist())
x = np.array(counts["DMY"].values.tolist())
addpersent(x,y, myMsg.shape[0], 0)
plt.title("Active hours:")
plt.legend()
plt.show()

14. Дослідити людей з якими в мене найбільше спільних чатів, каналів та їх кількість. 

In [ ]:
def getUsersIdFromDF(data):
    userlist = []
    for user in data["users"].values:
        listUser = list(map(int, re.findall(r'\d+', user)))
        if len(listUser) > 0:
            userlist.append(listUser[0])
    return userlist

users = pd.DataFrame()
users["amount"] = df_meta.groupby("users")["users"].count()
users.reset_index(inplace=True)
users = users.sort_values(by = "amount", ascending=False)
# you need to find yourself and drop
i = 0
for u in getUsersIdFromDF(users):
    if u == myUserId:
        users = users.drop([i])#drop myself
    i +=1    
    


condition = df_meta.isin({"users": myClassMates})["users"]
noClMt = pd.DataFrame()
noClMt["amount"] = df_meta[~condition].groupby("users")["users"].count()
noClMt.reset_index(inplace=True)
noClMt = noClMt.sort_values(by = "amount", ascending=False)

fig = plt.figure()
sub1 = fig.add_subplot(1,2,1)
sub2 = fig.add_subplot(1,2,2)


select = users.head(10)
xticks = getUsersIdFromDF(select)
y = np.array(select["amount"].values)
x = np.arange(len(xticks))
plt.sca(sub1)
plt.xticks(x, xticks)
sub1.bar(x, y)
plt.title("Comon groups with classmates")
plt.xticks(rotation=45, ha='right')

select = noClMt.head(10)
xticks = getUsersIdFromDF(select)
y = np.array(select["amount"].values)
x = np.arange(len(xticks))
plt.sca(sub2)
plt.xticks(x, xticks)
sub2.bar(x, y)
plt.title("Comon groups without classmates")
plt.xticks(rotation=45, ha='right')

plt

15. Дослідити частоту моїх повідомлень в чатах та групах відносно всіх повідомлень в них. Тобто 10% в групах означає що ви надсилаєте в середньому 10% повідомлень у всіх групах.

In [ ]:
myMsg = mergedDf[mergedDf["from_id"] == f"PeerUser(user_id={myUserId})"]
counts = pd.DataFrame()
counts["amount"] = myMsg.groupby("type_y")["type_y"].count()
counts["amount"].plot(kind = "bar", color = "orange")
counts.reset_index(inplace=True)
y = np.array(counts["amount"].values.tolist())
x = np.array(counts["type_y"].values.tolist())

plt.text(0,y[0],f'{round(y[0]/(mergedGroup.shape[0])*100, 1)}%')
plt.text(1,y[1],f'{round(y[1]/(mergedPrivate.shape[0])*100, 1)}%')

plt.title("My activity in chats")
plt.legend()

16. Дослідити людей яким я відправляла найбільшу кількість слів, та від яких отримувала.

In [ ]:
def putNumberOfWords(data):
    wordsAmount = []
    for ms in data["message"].values:
        mslist = re.sub("[^a-zA-Z\u0401-\u04f9]+", " ",str(ms).lower()).replace("[^a-zA-Z0-9]+", ' ').split(' ')
        mslist = list(filter(lambda a: a != [], mslist))
        if len(mslist)>0:
            wordsAmount.append(len(mslist))
    return wordsAmount

txtDF = mergedPrivate[mergedPrivate["type_x"]=="text"]
txtDF["words_amount"] = putNumberOfWords(txtDF)

mytxt = txtDF[txtDF["from_id"] == f"PeerUser(user_id={myUserId})"]
othertxt = txtDF[txtDF["from_id"] != f"PeerUser(user_id={myUserId})"]

recDF = pd.DataFrame()
sendDF = pd.DataFrame()
sendDF["words_amount"] = mytxt.groupby("name")["words_amount"].sum().sort_values(ascending=False).head(12)
recDF["words_amount"] = othertxt.groupby("name")["words_amount"].sum().sort_values(ascending=False).head(12)
recDF.reset_index(inplace=True)
sendDF.reset_index(inplace=True)


fig = plt.figure()
sub1 = fig.add_subplot(1,2,1)
sub2 = fig.add_subplot(1,2,2)


select = sendDF
xticks = select["name"].values
y = np.array(select["words_amount"].values)
x = np.arange(len(xticks))
plt.sca(sub1)
plt.xticks(x, xticks)
sub1.bar(x, y)
plt.title("Send amount of words")
plt.xticks(rotation=45, ha='right')

select = recDF
xticks = select["name"].values
y = np.array(select["words_amount"].values)
x = np.arange(len(xticks))
plt.sca(sub2)
plt.xticks(x, xticks)
sub2.bar(x, y)
plt.title("Received amount of words")
plt.xticks(rotation=45, ha='right')

plt

17. Дослідити використання слів паразитів. Результат поданий у частці повідомлень що містять погані слова

In [ ]:
poorwords = ["капець", "ок", "треш", "добре", "охх", "ммм", "ууу", "супер", "ужас", "клас", "короче", "слухай", "бля", "дивись", "ну", "ясно", "хз"]
def searcher(d, searchlist):
    column = []
    for ms in list(d["message"]):
        res = 0
        for w in searchlist:
            if w in str(ms).lower():
                res += 1
        if res>0:
            res = 1
        column.append(res)
    return column

dfr = df[df["from_id"] != f"PeerUser(user_id={myUserId})"]
dfr["DMY"] = convert_month(dfr)
dfrg = pd.DataFrame()
dfrg["all_words"] = dfr.groupby("DMY")["DMY"].count()
dfrg.reset_index(inplace=True)
dfs = df[df["from_id"] == f"PeerUser(user_id={myUserId})"]
dfs["DMY"] = convert_month(dfs)
dfsg = pd.DataFrame()
dfsg["all_words"] = dfs.groupby("DMY")["DMY"].count()
dfsg.reset_index(inplace=True)


pwr = mergedPrivate[mergedPrivate["from_id"] != f"PeerUser(user_id={myUserId})"]
pwr["DMY"] = convert_month(pwr)
pwr["Numb"] = searcher(pwr, poorwords)
pwrg = pd.DataFrame()
pwrg["poor_words"] = pwr.groupby("DMY")["Numb"].sum()
pwrg.reset_index(inplace=True)
pwrg = pwrg.merge(dfrg,  how='inner', on='DMY' )
pwrg["frequency"] = pwrg["poor_words"]/pwrg["all_words"]


pws = mergedPrivate[mergedPrivate["from_id"] == f"PeerUser(user_id={myUserId})"]
pws["DMY"] = convert_month(pws)
pws["Numb"] = searcher(pws, poorwords)
pwsg = pd.DataFrame()
pwsg["poor_words"] = pws.groupby("DMY")["Numb"].sum()
pwsg.reset_index(inplace=True)
pwsg = pwsg.merge(dfsg,  how='inner', on='DMY' )
pwsg["frequency"] = pwsg["poor_words"]/pwsg["all_words"]

fig = plt.figure()
sub1 = fig.add_subplot(211)

pwrg.plot(ax = sub1, x = "DMY", y = "frequency", color = "black", label = "received")
pwsg.plot(ax = sub1, x = "DMY", y = "frequency", color = "red", label = "sent")
plt.sca(sub1)
plt.title("Using poor words")
plt.legend()
plt.show()

18. 10 найчастіших слів, у спілкуванні із партнером, що довші за 3 символи.

In [ ]:
chWHPartnerName = input("Your chat name of partner or friend name dialog")

In [ ]:
MIN_LENGTH = 4
TOP = 10
def getAllWordFrq(name):
    wordsDict = {}
    d = mergedPrivate[mergedPrivate["name"] == name]["message"]
    for ms in d.values:
        mslist = re.sub("[^a-zA-Z\u0401-\u04f9]+", " ",str(ms).lower()).replace("[^a-zA-Z0-9]+", ' ').split(' ')
        mslist = list(filter(lambda a: len(a) >= MIN_LENGTH and a not in ukr_stopwords, mslist))
        for w in mslist:
            if w in wordsDict:
                wordsDict[w]+=1
            else:
                wordsDict[w]=1
    return wordsDict

def getTop(wordDict):
    tups = sorted(wordDict.items(), key=lambda kv: kv[1], reverse= True)
    res = pd.DataFrame(columns = ["word", "amount"])
    for i in range(TOP):
        res.at[i, 'word'] = tups[i][0]
        res.at[i,'amount'] = tups[i][1]
    return res
    
    
    
top10 = getTop(getAllWordFrq(chWHPartnerName))
xticks = top10["word"].values
y = np.array(top10["amount"].values)
x = np.arange(len(xticks))
plt.xticks(x, xticks)
plt.bar(x, y)
addlabels(x, y)
plt.xticks(rotation=45, ha='right')
plt.title("The most popular words with my bf")
plt.show()  

19. Середня довжина повідомлення по датам

In [ ]:
def getMsgLen(data):
    msgLens = []
    for ms in data["message"].values:
        mslist = str(ms).lower().replace("[^\u0041-\u007A\u0430–\u0456]+", ' ').split(' ')
        mslist = list(filter(lambda a: len(a) > 0, mslist))
        msgLens.append(len(mslist))
    return msgLens

mytxt["DMY"] = convert_month(mytxt)
othertxt["DMY"] = convert_month(othertxt)
mytxt["length"] = getMsgLen(mytxt)
othertxt["length"] = getMsgLen(othertxt)

othertxt.groupby("DMY")["length"].mean().plot(label = "recieved")
mytxt.groupby("DMY")["length"].mean().plot(label = "send")
plt.legend()
plt.title("Average amount of words in message")

20. Частота використання не текстової взаємодії: реакцій, пересилання повідомлень та інших видів повідомлень.

In [ ]:
def getNumbFwd(d):
    res = 0
    for ms in d["fwd_from"].values:
        if str(ms) != "nan":
            res += 1
    return res
def getNumbReact(d):
    res = 0
    for ms in d["reactions"].values:
        if str(ms) != "{}":
            res += 1
    return res
        
    
allms = dfr.shape[0]    

fig = plt.figure()
sub1 = fig.add_subplot(1,2,1)
sub2 = fig.add_subplot(1,2,2)


forward = getNumbFwd(dfr) 
reactions = getNumbReact(dfr) 
notText = dfr[dfr["type"] != "text"].shape[0]

xticks = ["Forwarded", "With reactions", "Not-text"]
y = np.array([forward, reactions, notText])
x = np.arange(len(xticks))
plt.sca(sub1)
plt.xticks(x, xticks)
sub1.bar(x, y)
addpersent(x,y, allms, 1)
plt.title("Kinds of recieved messanges")
plt.xticks(rotation=30, ha='right')

forward = getNumbFwd(dfs) 
reactions = getNumbReact(dfs) 
notText = dfs[dfs["type"] != "text"].shape[0]

y = np.array([forward, reactions, notText])
plt.sca(sub2)
plt.xticks(x, xticks)
sub2.bar(x, y)
addpersent(x,y, allms, 1)
plt.title("Kinds of send messanges")
plt.xticks(rotation=30, ha='right')
plt.show() 


21. Порівняння використання латиниці та кирилиці. Ймовірна причина росту використання латиниці - запрошення в зум-конференцію, що генеруються англійською.

In [ ]:
def getMsgLeng(data, leng = "ua"):
    msgLens = []
    rule = "[^\u0430–\u0456]+"
    if leng =="en":
        rule = "[^\u0041-\u007A]+"
    for ms in data["message"].values:
        mslist = re.sub(rule, ' ', str(ms).lower()).split(' ')
        mslist = list(filter(lambda a: len(a) > 0 and a!='nan'and a!='–', mslist))
        msgLens.append(len(mslist))
    return msgLens

In [ ]:
mytxt["ua"] = getMsgLeng(mytxt)
othertxt["ua"] = getMsgLeng(othertxt)
mytxt["en"] = getMsgLeng(mytxt, leng = "en")
othertxt["en"] = getMsgLeng(othertxt, leng = "en")

fig = plt.figure()
sub1 = fig.add_subplot(211)
sub2 = fig.add_subplot(212)

othertxt.groupby("DMY")["en"].sum().plot(ax = sub1, label = "recieved en")
mytxt.groupby("DMY")["en"].sum().plot(ax = sub1, label = "send en")
othertxt.groupby("DMY")["ua"].sum().plot(ax = sub2, label = "recieved ua")
mytxt.groupby("DMY")["ua"].sum().plot(ax = sub2, label = "send ua")
plt.sca(sub1)
plt.title("Amount of English words")
plt.legend()
plt.sca(sub2)
plt.title("Amount of  Ukrainian words")
plt.legend()

22. Sentiment analysis. Average tone of message

In [ ]:
tone_dict = {}
with open("tone-dict-uk-full.csv", encoding="utf-8") as file:
    for line in file:
        (key, value) = line.replace('\n', '').split(",")
        try:
            tone_dict[key.lower()] = float(value)
        except:
            pass

In [ ]:
def getMsgTone(data):
    msgTone = []
    for ms in data["message"].values:
        mslist = re.sub("[^a-zа-яі]+", ' ', str(ms).lower()).split(' ')
        mslist = list(filter(lambda a: len(a) > 0 and a!='nan'and a!='–', mslist))
        score = 0
        for w in mslist:
            if w in tone_dict:
                score+=tone_dict[w]           
        msgTone.append(score)
    return msgTone

In [ ]:
mytxt["tone"] = getMsgTone(mytxt)
othertxt["tone"] = getMsgTone(othertxt)

othertxt.groupby("DMY")["tone"].mean().plot(label = "recieved")
mytxt.groupby("DMY")["tone"].mean().plot(label = "send")
plt.legend()
plt.title("Average message tone")

23. Знаходження часу на добу, протягом якаго я можу надсилати повідомлення - середня різниця між першим та останім повідомленням в день

In [ ]:
from datetime import timedelta
from datetime import datetime
def convert_itimes(d, hours):
    mylist = []
    for t in range(len(d["date"].tolist())):
        msdate = d["date"].loc[d.index[t]][:19]
        mylist.append(str(datetime.strptime(msdate, "%Y-%m-%d %H:%M:%S")+timedelta(hours = hours)))
    return mylist

def convert_iminutes(d):
    mylist = []
    for t in range(len(d["itime"].tolist())):
        msghour = int(d["itime"].loc[d.index[t]][11:13])
        msgminutes = int(d["itime"].loc[d.index[t]][11:13])+60*msghour
        mylist.append(msgminutes)
    return mylist

def convert_idays(d):
    mylist = []
    for t in range(len(d["itime"].tolist())):
        mylist.append(pd.to_datetime(d["itime"].loc[d.index[t]][:10]))
    return mylist


def calcut_diff(d):
    res = {}
    for dmy in d["DMY"].tolist():
        if not dmy in res:
            res[dmy] = float(d[d["DMY"] == dmy]['minuts'].max() - d[d["DMY"] == dmy]['minuts'].min())/60
    return res

In [ ]:
dfs["itime"] = convert_itimes(dfs, -3)

dfs["minuts"] = convert_iminutes(dfs)
dfs["DMY"] = convert_idays(dfs)

In [ ]:
res = sorted(calcut_diff(dfs).items())
x, y = zip(*res)
plt.scatter(x, y, color = "purple", alpha = 0.3)
plt.xticks(rotation=45, ha='right')
plt.title("Active amount of hours per day")

24. find topics of one day chatting with one user using TD-IDF

In [ ]:
def concat_docs(d):
    mslists = d.groupby(["dialog_id", "DMY"])["message"].apply(list).tolist()
    res = []
    for l in mslists:
        l = [item for item in l if not pd.isnull(item)]
        res.append(' '.join(l).lower())
    return [item for item in res if item != ' ']

        

In [ ]:
privates = mergedPrivate
privates["DMY"] = convert_times(privates)
documents = concat_docs(privates)

In [ ]:
import spacy 

nlp = spacy.load("uk_core_news_sm")

In [ ]:
lemdocs = []
listlems = []
for doc in documents:
    tokens = nlp(doc)
    text = []
    for token in tokens:
        text.append(token.lemma_)
    l = [item for item in text if len(item) > 1 and re.match(r'^[а-яА-ЯіІ]+$', item) and item not in ukr_stopwords]
    if len(l)>0:
        lemdocs.append(" ".join(l))
        listlems.append(l)




In [ ]:
#!pip install scikit-learn

In [ ]:
import sklearn as sl
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(lemdocs)
dftfidf = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names_out(), columns=["TF-IDF"])
dftfidf = dftfidf.sort_values('TF-IDF', ascending=False)
dftfidf.head(25)

In [ ]:
topics = []
for lems in listlems:
    toplem = lems[0]
    topi = 0.0
    seclem = lems[-1]
    seci = 0.0
    for lem in lems:
        if float(dftfidf.loc[lem])>topi:
            seclem = toplem
            seci = topi
            topi = float(dftfidf.loc[lem])
            toplem = lem
    topics.append(f"{toplem} {seclem}")
topics[:10]

25. За допомогою TF-IDF аналізу, визначити найпопулярніші теми на які я спілкуюсь.

In [ ]:
from collections import Counter
top_topics = [key for key, value in Counter(topics).most_common()]
top_topics[:20]